# 2.3 优化提示词改善答疑机器人回答质量

## 🚄 前言

上一节的答疑机器人已经可以回答关于公司规章制度的问题，但这仅仅是构建优秀用户体验的第一步。用户往往期待更个性化、更精准的交互。  
本节将介绍提示词工程的技巧，教你如何通过优化提示词来改善或控制答疑机器人的输出，例如调整语气、规范格式，甚至赋予它处理文本总结、推断、转换等多种任务的能力。

## 🍁 课程目标

学完本课程后，你将：

* 了解提示词框架和模版
* 了解提示词技巧和最佳实践
* 学习在工程中应用大模型处理多种不同类型的任务

## 1. 前文回顾

上一节中通过RAG方法，大模型已经获取到了公司的私有知识。为了方便调用，将其封装成了几个函数，并保存在了 chatbot/rag.py 中。 现在你可以通过如下代码来快速调用。

In [ ]:
from chatbot import rag
import os
from config.load_key import load_key

In [4]:
# 加载API密钥
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


In [5]:
# 加载索引
# 上小节已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index()
query_engine = rag.create_query_engine(index=index)

In [6]:
# 定义问答函数
def ask_llm(question, query_engine):
  streaming_response = query_engine.query(question)
  streaming_response.print_response_stream()

## 2. 优化提示词以改善大模型回答质量

上一节中，你通过RAG方法获得了公司内部使用的管理工具。但你的同事希望答疑机器人不仅要给出工具名称，还需要附带工具链接。虽然可以让同事在提问时加上“给出下载地址”的说明，但是这种做法并不高效。因此你考虑改进程序：在收到用户问题后，补充一些对回答内容的要求。

In [7]:
question = "我们公司项目管理应该用什么工具"
instruction = "如果是工具咨询类问题，请务必给出下载地址链接。"
new_question = question + instruction
ask_llm(new_question,query_engine)

对于项目管理，可以使用Jira或Trello这样的工具。这些工具能够帮助团队有效地跟踪项目的进度，管理任务，并确保所有成员都对项目的当前状态有清晰的了解。例如，Jira非常适合软件开发项目，而Trello则以其灵活性著称，适用于多种类型的项目管理。您可以通过访问它们的官方网站来下载和安装这些工具：[Jira](https://www.atlassian.com/software/jira) 和 [Trello](https://trello.com/)。

调整了提示的方式后，答疑机器人的回答更能符合同事的要求。所以提示词在很大程度上决定了大模型的回答质量，接下来你可以参考一些提示词框架构建提示词。

## 3. 提示词框架

### 3.1 基本要素

当和大模型在交流时，可以将它想象是一个经过“社会化训练的”人，交流方式应当和人与人之间传递信息的方式一样。你的需求需要清晰明确，不能有歧义。你的提问方式（Prompt）越清晰明确，大模型越能抓住问题的关键点，回复就越符合你的预期。一般来说，提示词中需要明确以下几个要素：**任务目标、上下文、角色、受众、样例、输出格式**。这些要素构成了一个提示词框架，能帮助你构建一个完整、有效的提示词。

|要素|含义|
|----|----|
|任务目标（Object）|明确要求大模型完成什么任务，让大模型专注具体目标|
|上下文（Context）|任务的背景信息，比如操作流水、任务场景等，明确大模型理解讨论的范围|
|角色（Role）|大模型扮演的角色，或者强调大模型应该使用的语气、写作风格等，明确大模型回应的预期情感|
|受众（Audience）|明确大模型针对的特定受众，约束大模型的应答风格|
|样例（Sample）|让大模型参考的具体案例，大模型会从中抽象出实现方案、需要注意的具体格式等信息|
|输出格式（Output Format）|明确指定输出的格式、输出类型、枚举值的范围。通常也会明确指出不需要输出的内容和不期望的信息，可以结合样例来进一步明确输出的格式和输出方法|

当然，除了上面讲的提示词框架，许多问题分析的思维范式都可以用来帮助你描述清晰具体的需求。例如，SWOT分析法、5W2H分析法等。另外，你也可以考虑使用阿里云百炼提供的[提示词自动优化工具](https://bailian.console.aliyun.com/#/prompt-manage)，来帮助你完善提示词。

<img src="https://img.alicdn.com/imgextra/i3/O1CN014JCCqn22zX6xB4tt3_!!6000000007191-0-tps-2068-1052.jpg" width="700"/>

### 3.2 提示词模板

在开发大模型应用时，直接让用户根据框架书写提示词并非最佳选择。你可以参考各种提示词框架中的要素，构建一个提示词模板。提示词模版可以预设部分信息，如大模型的角色、注意事项等，以此来约束大模型的行为。开发者只需在模板中配置输入参数，便能创建标准化的大模型的应用。

使用 LlamaIndex 中创建的 RAG应用中，有个默认的提示词模板，如下所示：

- 默认的模版可以使用代码查看，你可以参考[LlamaIndex官网的代码。](https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/)

```text
LlamaIndex原始prompt模板为
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
```

其中，`context_str`和`query_str`都表示变量。在进行向量检索和提问过程中，`context_str`将替换为从向量库中检索到的上下文信息，`query_str`则替换为用户的问题。

由于原始模版是通用模版，不适合用来约束答疑机器人的行为。你可以通过下列示例代码重新调整提示词模版，其中`prompt_template_string`表示新的提示词模版，你可以根据自己的场景自行修改。

In [8]:
# 构建提示词模版
prompt_template_string = (
    "你是公司的客服小蜜，你需要简明扼要的回答用户的问题"
    "【注意事项】：\n"
    "1. 根据上下文信息而非先验知识来回答问题。\n"
    "2. 如果是工资、公司的工程代码、项目报价等涉及隐私的问题，请你提醒用户：对不起，您的问题触及安全红线，无法提供\n"
    "3. 你只需要回答用户的问题，不要输出其他信息\n"
    "以下是参考信息。"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "问题：{query_str}\n。"
    "回答："
)

# 更新提示词模版
rag.update_prompt_template(query_engine,prompt_template_string)

## 4. 构建有效的提示词技巧

在3.1中列举了一些提示词设计中的要素，接下来将从提示词要素出发，结合具体场景展开讲解提示词技巧。

### 4.1 清晰表达需求，并使用分隔符

明确的表达需求可以确保大模型生成的内容与任务高度相关。需求包括任务目标、背景及上下文信息，还可以使用分隔符将各种提示词要素隔开。

分隔符可以使大模型抓住具体的目标，避免模糊的理解，也减少对不必要信息的处理。分隔符一般可以选择 “**【】**”、“**<< >>**”、“**###**”来标识关键要素，用“**===**”、“**---**”来分隔段落，或者使用xml标签如`<tag> </tag>`来对特定段落进行标识。当然，分隔符不止上述提到的几种，只需要起到明确阻隔的作用即可。需要注意的是，如果提示词中已大量使用某种符号（如【】），则应避免用该符号作为分隔符，以防混淆。

在下面的示例中，将基于3.2中的提示词模版，以答疑机器人的身份帮你完成文档润色。

- 你需要运行3.2中的代码，才能使新的提示词模版生效。

In [20]:
question = """
把下列【】括起来的文本进行扩写和润色，让文案生动且富有创造力，并且受到公司新员工的喜欢。
【新员工训练营活动】
"""
ask_llm(question,query_engine)

【新员工训练营活动】欢迎加入我们的大家庭！为了让每位新成员都能快速融入团队，我们精心策划了“新员工训练营”活动。这不仅是一次简单的入职培训，更是一场充满挑战与乐趣的旅程。在这里，你将与来自不同部门的小伙伴们一起，通过一系列互动游戏和专业课程，了解公司的文化、价值观以及工作流程。我们相信，通过这次训练营，你不仅能学到宝贵的知识和技能，还能结识一群志同道合的朋友，共同开启职业生涯的新篇章。让我们携手并进，共创美好未来！

由上述运行结果可以发现，提示词中明确了任务需求：对文本进行扩写和润色，任务主题为“**新员工训练营活动**”，受众为“**公司员工**”，并使用了“**【】**”分隔符分隔了文本。使用这样的提示词，让输出的文本不仅保持了原本的含义，还更加生动有趣。

### 4.2 限定角色和受众

角色指大模型在特定场景下应扮演的身份，如专家、顾问或助手。受众是使用模型输出的目标用户群体，如普通消费者或学生。这有助于开发者定制内容的风格和深度。以下示例将展示不同的系统角色对输出文本风格和内容的影响，用户的问题为：qwem-vl是什么？

- 以下仅演示提示词模版和输出的内容，你可以将提示词模版传入到`prompt_template_string`中，将用户问题传入到`question`中，再调用`ask_llm`函数。

<table>
  <thead>
    <tr>
      <th width = '80px'>角色</th>
      <th width = '500px'>大模型算法工程师</th>
      <th width = '500px'>小学老师</th>
    </tr>
  </thead>
  <tbody valign = 'top'>
    <tr>
      <td>提示词模版</td>
      <td>"你是资深的大模型算法工程师，请根据用户的问题进行解答\n"<br>"---------------------\n"<br>"{context_str}\n"<br>"---------------------\n"<br>"问题：{query_str}\n。"<br>"回答："</td>
      <td>"你是一名小学老师，请根据用户的问题进行解答\n"<br>"---------------------\n"<br>"{context_str}\n"<br>"---------------------\n"<br>"问题：{query_str}\n。"<br>"回答："</td>
    </tr>
    <tr>
      <td>用户问题</td>
      <td>Qwen-VL</td>
      <td>Qwen-VL</td>
    </tr>
    <tr>
      <td>输出回答</td>
      <td>Qwen-VL是阿里巴巴云推出的一种多模态预训练模型，它结合了文本和图像处理能力，能够理解和生成与图像相关的文本描述。简单来说，Qwen-VL可以看作是一个能够“看图说话”的人工智能助手，它不仅能够理解图片的内容，还能根据图片内容生成相应的文字描述或回答相关问题。这种技术在多个领域都有广泛的应用前景，比如自动图像标注、视觉问答系统、基于内容的图像搜索等。对于新媒体运营而言，Qwen-VL这样的工具可以帮助提高内容创作效率，增强用户体验，为用户提供更加丰富多元的信息展示方式。</td>
      <td>小朋友们，我们今天来认识一个新朋友，它的名字叫做Qwen-VL！<br>Qwen-VL就像是一个非常聪明的机器人，它拥有“眼睛”👀和“大脑”🧠。它能“看”图片、视频，还能理解它们的意思，就像我们一样！<br>比如，你给它看一张小猫玩毛线球的图片，它不仅能认出这是一只小猫🐱，还能告诉你它在玩毛线球🧶。 更厉害的是，你还可以问它：“小猫是什么颜色的？”，它就能告诉你小猫的颜色。<br>它还能理解视频里的内容，比如你给它看一段小朋友们踢足球⚽️的视频，它能告诉你谁在踢球，球踢到了哪里，甚至还能猜到接下来可能会发生什么！<br>Qwen-VL就像一个正在学习的小朋友，它还在不断地学习和进步，将来它会变得更聪明，能做更多的事情，帮助我们更好地了解这个世界！</td>
    </tr>
  </tbody>
</table>


### 4.3 规定输出格式
有时候，开发者在设计大模型应用时，需要采用结构化的数据作为下游系统的输入，才能完成整个应用的开发，但是一般大模型是输出连续的文本。不用担心，大模型有结构化输出的能力。你只需在提示词中指定输出的格式，大模型有很大可能会输出结构化的内容。

在下面的示例中，将基于3.2中的提示词模版，以答疑机器人的身份帮助公司开发的文档进行文档质量审查，并将结果以json的格式输出：

In [9]:
question_task= """
【任务要求】
你将看到一句话或一段话。你需要审查这段话中有没有错别字。如果出现了错别字，你要指出错误，并给出解释。
 “的” 和 “地” 混淆不算错别字，没有错误
---
【输出要求】
请输出json格式
其中，label只能取0或1，0代表有错误，1代表没有错误
reason是错误的原因
correct是修正后的文档内容
不要输出其他内容
---
【用户输入】
以下是用户输入，请审阅：
"""
question_doc = "分隔符是特殊的符号，它们帮助大语言膜形 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"

question = question_task + question_doc
ask_llm(question, query_engine)

```json
{
  "label": 0,
  "reason": "‘膜形’应为‘模型’",
  "correct": "分隔符是特殊的符号，它们帮助大语言模型 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"
}
```

由上述示例的结果可知，在提示词`question_task`中注明了输出格式为json，并且在输出中大模型也指出了文档的错误，格式为json，但是这样的输出并不能被下游任务所使用，因为还多输出了` ```json xxx ``` `的内容。虽然可以通过对字符串的处理，删除多余的字符，但是为了减少代码维护的成本，并确保整个系统的稳定性，你可以在提示词中添加部分样例，这个方法将在下面介绍。

### 4.4 提供少样本示例

由4.3的结果可知，大模型无法理解所需的输出格式。既然大模型还不知道输出格式，那么先给它提供几个样例，给它当“参考书”。在下列代码中，将对4.3中的示例进行优化，在提示词中添加两个找错别字的例子，包括输入和输出。接下来，观察优化提示词后答疑机器人的输出吧！

In [22]:
question_task= """
【任务要求】
你将看到一句话或一段话。你审查这段话中有没有错别字。如果出现了错别字，你要指出错误，并给出解释。
 “的” 和 “地” 混淆不算错别字，没有错误
---
【输出要求】
请输出json格式
其中，label只能取0或1，0代表有错误，1代表没有错误
reason是错误的原因
correct是修正后的文档内容
不要输出其他内容
---
【样例】

输入：逛合作用是植物、藻类和某些细菌利用光能将二氧化碳和水转化为有机物和氧气的过程
输出：{"label":0, "reason":"出现了'逛合作用'文字错误，正确应为'光合作用'", "correct":"光合作用是植物、藻类和某些细菌利用光能将二氧化碳和水转化为有机物和氧气的过程。"}

输入：阳光真好，我们去散步吧。
输出：{"label":1, "reason":"没有错误", "correct":"阳光真好，我们去散步吧。"}
【用户输入】
以下是用户输入，请审阅：
"""

question_doc = "分隔符是特殊的符号，它们帮助大语言膜形 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"

question = question_task + question_doc
ask_llm(question, query_engine)

{"label":0, "reason":"出现了'大语言膜形'文字错误，正确应为'大语言模型'", "correct":"分隔符是特殊的符号，它们帮助大语言模型 (LLM) 识别提示中哪些部分应当被视为一个完整的意思单元。"}

你可以多次运行上述代码，发现大模型不再输出` ```json  xxx ``` `这种无关内容了，只有json格式数据。这种稳定性的格式化的输出，使得在现有的系统中接入大模型这个操作变得具有可行性。

在新闻网站、博客平台或企业内部的知识分享平台上，用户编辑或发布的文章可能会包含错别字、语法错误、甚至是敏感信息。但是传统的人工审核方式很容易出疏漏。这时候可以接入大模型来对内容做审查工作。如果文章被标记为存在严重语法错误或含有高风险敏感词汇，则将其修改的优先级设置为“高”。对于轻微问题的文章，则可以将其修改的优先级设置为“低”。这样会节省人力成本，提高系统的效率与准确性。

当然了，类似于上述场景的应用非常多，开发者可以分析系统中流程的瓶颈或者关注数据密集型的任务，探索更多的大模型应用场景。

### 4.5 给模型“思考”的时间

对于一些复杂的任务来说，使用上面提到的提示词也许还不能帮助大模型完成任务。但是你可以通过让大模型一步步“思考”，引导大模型输出任务的中间步骤，允许大模型在进行推理之前，得到更多的依据，从而提升在复杂任务的表现能力。思维链（COT）方法是让模型进行思考的一种方法。它通过让模型处理中间步骤，逐步将复杂问题分解为子问题，最终推导出正确答案。

假设有这样的场景，让大模型计算下面这道数学题，在此先提示一下，这道题的正确答案为10500元。先使用简单的提示词：

In [23]:
question = """
【背景信息】
某教育培训机构（以下简称“公司”）在2023年度发生了以下主要支出：
为了给不同城市的学校学生上课，公司的老师全年共出差了5次，每次出差时间为一周，具体费用如下：
   - 交通费及住宿费：平均1600元/次
   - 教学用具采购费用：公司在年初一次性购买了一批教学用具，总价为10000元，预计可以使用4年。
   
【问题描述】
请根据上述背景信息，完成以下任务：
计算全年因教师出差而产生的差旅总费用，包括摊销的教学用具。

【输出要求】
直接给出总差旅费用，不要其他信息"""

ask_llm(question, query_engine)

9200元

由上面的实验结果来看，大模型计算结果不正确。下面将使用COT方法，让大模型逐步进行思考。

In [24]:
question = """某教育培训机构（以下简称“公司”）在2023年度发生了以下主要支出：
为了给不同城市的学校学生上课，公司的老师全年共出差了5次，每次出差时间为一周，具体费用如下：
   - 交通费及住宿费：平均1600元/次
   - 教学用具采购费用：公司在年初一次性购买了一批教学用具，总价为10000元，预计可以使用4年。
   
### 问题描述
请根据上述背景信息，完成以下任务：
计算全年因教师出差而产生的差旅总费用，包括摊销的教学用具。

### 输出要求
请你一步步推导，计算总差旅费用"""

ask_llm(question, query_engine)

好的，根据您提供的信息，我们可以分步骤来计算全年的差旅总费用，包括摊销的教学用具费用。

1. **计算交通费及住宿费**：
   - 每次出差的交通费及住宿费为1600元。
   - 全年共出差5次。
   - 因此，交通费及住宿费总计为：1600元/次 × 5次 = 8000元。

2. **计算教学用具的年摊销费用**：
   - 教学用具总价为10000元，预计使用4年。
   - 年摊销费用为：10000元 ÷ 4年 = 2500元/年。

3. **计算总差旅费用**：
   - 总差旅费用 = 交通费及住宿费 + 教学用具年摊销费用
   - 总差旅费用 = 8000元 + 2500元 = 10500元。

因此，全年因教师出差而产生的差旅总费用为10500元。

经过优化后的提示词，大模型能够准确计算出结果。因此在开发大模型应用时，可以在提示词中添加思维链的方法，可以确保一些推理任务能正确执行。  

使大模型进行 “思考”的方法还有很多种，比如：思维树（ToT)、思维图（GOT） 等。但是就目前大模型的发展来说，仅靠引导大模型“思考”还是无法完成更复杂的工作。大模型也逐渐从COT的提示方法向多智能体（Agent）方向进行发展。你可以在<2.6 用插件扩展答疑机器人的能力边界>阅读详细内容。

## 5. 使用大模型进行意图识别

经过学习了提示词技术，同事们了解了答疑机器人不仅可以用来查资料，还可以用来帮助他们审查文档、翻译文档。尽管大多数情况下，你的答疑机器人是可以帮助发现问题的，但答疑机器人仍存在一些不足。如下示例所示：


In [25]:
rag.ask('请帮我审查下这句话：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？', query_engine=query_engine)

是的，内容开发工程师需要设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等，确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，还会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。

由上面的实验结果可知，答疑机器人直接对这段话进行了回答，并没有理解你需要它对这句话进行审查的意图。下面来查看下，答疑机器人在回答之前从知识库中召回了哪些内容，下列仅展示召回的部分内容：

- 你可以在<2.4 自动化评测答疑机器人的表现>中，了解如何查看从知识库召回的片段。

```json
片段: 内容开发工程师
......
1. 内容研究与分析
对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。

2. 教材和课程开发
根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。
......
```

从知识库召回的片段可知，大部分内容都涉及技术内容工程师的工作职责，可以认为答疑机器人被知识库中的上下文干扰了，导致其回答的内容偏离了文档审查的主题。

既然上下文信息影响了大模型的回答，那么就不必每次提问都使用RAG方法。你可以借助大模型先进行意图识别：对用户的问题进行分类，如果是文档审查、内容翻译等任务，会直接输入给大模型来生成答案，如果是内部知识查询问题，才经过RAG 链路来生成答案。

<img src="https://img.alicdn.com/imgextra/i1/O1CN0126zKe71PAuJjWfQ3N_!!6000000001801-0-tps-2254-1080.jpg" width="800">

让大模型进行意图识别也有以下两种方法： 

- 使用提示词：通过设计特定的提示词，引导大模型生成符合预期的回答。这种方法不需要修改模型本身的参数，而是依靠构造的输入来激发模型内部已有的知识。
- 对模型进行微调：基于一个经过预训练的基础模型，使用特定的标注数据进一步训练该模型，使其更好的对意图进行分类。微调涉及调整模型的部分或全部参数。

在本节中，将借助书写有效的提示词帮助大模型进行意图识别，通过微调的方法你可以在<2.8 通过微调提升模型的准确度与效率>小节进行学习。

### 5.1 意图识别

接下来，将构建提示词使大模型对问题分类。由于经过意图识别后要取得格式化的内容，才能进行文档审查或者使用RAG应用，所以为了能将用户的问题准确分类，将考虑以下提示词技巧：
- 明确输出格式：指定输出格式，使分类结果规范且易于解析。
- Few-shot 示例：提供示例，帮助大模型理解每个类别的特征和分类规则。

In [26]:
from chatbot import llm
  
# 构建提示词
prompt = '''
【角色背景】
你是一个问题分类路由器，负责判断用户问题的类型，并将其归入下列三类之一：
1. 公司内部文档查询
2. 内容翻译
3. 文档审查

【任务要求】
你的任务是根据用户的输入内容，判断其意图并仅选择一个最贴切的分类。请仅输出分类名称，不需要多余的解释。判断依据如下：

- 如果问题涉及公司政策、流程、内部工具或职位描述与职责等内容，选择“公司内部文档查询”。
- 如果问题涉及任意一门非中文的语言，且输入中没有任何出现任何外语或出现“翻译”等字眼，选择“内容翻译”。
- 如果问题涉及检查或总结外部文档或链接内容，选择“文档审查”。
- 用户的前后输入与问题分类并没有任何关系，请单独为每次对话考虑分类类别。

【Few-shot 示例】
示例1：用户输入：“我们公司内部有哪些常用的项目管理工具？”
分类：公司内部文档查询

示例2：用户输入：“请翻译下列句子：How can we finish the assignment on time?”
分类：内容翻译

示例3：用户输入：“请审查下这个链接下的文档：https://help.aliyun.com/zh/model-studio/user-guide/long-context-qwen-long”
分类：文档审查

示例4：用户输入：“请审查以下内容：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？”
分类：文档审查

示例5：用户输入：“技术内容工程师核心职责是什么？”
分类：公司内部文档查询

【用户输入】
以下是用户的输入，请判断分类：
'''

# 获取问题的类型
def get_question_type(question):
    return llm.invoke(prompt + question)

print(get_question_type('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')
print(get_question_type('That is a big one I dont know why'),'\n')
print(get_question_type('作为技术内容工程师有什么需要注意的吗？'),'\n')

文档审查 

内容翻译 

公司内部文档查询 



通过明确的输出格式和 few-shot 示例，答疑机器人可以更准确地识别问题类型并输出符合预期的格式。这种优化让分类任务更加标准化，为接下来添加意图识别到答疑机器人中打下了基础。


### 5.2 将意图识别应用到答疑机器人中

对用户的问题进行意图识别后，你就可以让答疑机器人先识别问题的类型，再使用不同的提示词和工作流程来回答问题。

In [27]:
def ask_llm_route(question):
    question_type = get_question_type(question)
    print(f'问题：{question}\n类型：{question_type}')
  
    reviewed_prompt = """
    【角色背景】
    你是文档纠错专家，负责找出文档中或网页内容的明显错误
    【任务要求】
    - 你需要言简意赅的回复。
    - 如果没有明显问题，请直接回复没有问题\n
    【输入如下】\n"""
  
    translate_prompt = """
   【任务要求】
    你是一名翻译专家，你要识别不同语言的文本，并翻译为中文.
    【输入如下】\n"""

    if question_type == '文档审查':
        return llm.invoke(reviewed_prompt + question)
    elif question_type == '公司内部文档查询':
        return rag.ask(question, query_engine=query_engine)
    elif question_type == '内容翻译':
        return llm.invoke(translate_prompt + question)
    else:
        return "未能识别问题类型，请重新输入。"

query_engine =rag.create_query_engine(index=rag.load_index())

In [28]:
# 问题1
print(ask_llm_route('https://www.promptingguide.ai/zh/techniques/fewshot'),'\n')

# 问题2
print(ask_llm_route('请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题3
print(ask_llm_route('技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'),'\n')

# 问题4:
print(ask_llm_route('A true master always carries the heart of a student.'),'\n')


问题：https://www.promptingguide.ai/zh/techniques/fewshot
类型：文档审查
没有问题。 

问题：请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：文档审查
没有问题。 

问题：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：公司内部文档查询
技术内容工程师确实需要进行内容优化与更新，这包括根据学习者的反馈和评价来识别并解决内容中的潜在问题，以及定期更新材料以反映新的研究成果、技术进步和市场变化。此外，他们也需要与多个部门紧密合作，比如教学设计师、教育心理学家、技术团队及市场营销人员，以确保内容的技术实施过程顺利，并有效传达给目标受众。这种跨部门的合作有助于共同创造出既有教育价值又具市场竞争力的产品。None 

问题：A true master always carries the heart of a student.
类型：内容翻译
一位真正的大师总是怀有一颗学生的心。 



从上述实验中可以看出，优化后的答疑机器人，并不是每次提问都会经过RAG链路，这不仅节省了资源，还避免了知识库的信息，干扰了大模型的推理，从而导致回答质量不好。
- 节省资源：对于检查文档错误的问题，大模型其实可以直接回复，并不需要检索参考资料，现在的实现存在资源浪费。
- 避免误解：因为现在每次会检索参考资料，这些被召回的相关文本段可能会干扰大模型理解问题，导致答非所问。

## ✅ 本节小结

通过学习本节课程，你已经掌握了提示词的基本框架和书写格式。此外，还学习了如何用提示词开发复杂的大模型应用。

实际落地大模型应用的过程中，提示词部分经常让领域专家来共同设计。因此，在你的工程代码中硬编码提示词，应该考虑调整成可配置的，甚至应用流程也可配置，这样能更方便领域专家参与提示词和整个流程的设计。阿里云百炼提供了可视化的大模型[应用构建](https://help.aliyun.com/zh/model-studio/user-guide/application-introduction#7c79befb2djg9)能力，可以在页面上完成提示词编写、整个复杂应用的流程可视化搭建，非常适合在需要非技术背景的领域专家参与的大模型应用开发项目中。

在下一节中，你将学习自动化评测方法测试答疑机器人的表现，你可以使用量化的指标来评测你的提示词的效果。

## 🔥 课后小测验

【单选题】 2.2.1 以下哪个提示词要素用于明确要求大模型完成的任务？（ ）

A. 角色 (Role)     
B. 受众 (Audience)   
C. 任务目标 (Object)  
D. 上下文 (Context)  
答案：C

解释： 任务目标 (Object) 明确规定了大模型需要执行的操作或达成的结果。其他选项并非直接定义任务本身。角色 (Role) 定义大模型扮演的身份，受众 (Audience) 定义目标群体，
上下文 (Context) 提供背景信息。


【多选题】 2.2.2. 假设你想使用大模型生成一段适合小学三年级学生阅读的关于太阳系的描述。以下哪个 user_query 的设计更合理？（ ）

A. user_query="Write about the solar system."    
B. user_query="Tell me about the solar system."     
C. user_query="Explain the solar system as if you were talking to a third-grade student."     
D. user_query="Write a short and engaging description of the solar system for a third-grade audience, focusing on key planets and their characteristics."   
E. user_query="太阳系是什么？"   

答案: C, D (这两个选项明确指定了目标读者，并对内容和风格提出了要求。)

## ✉️ 评价反馈

感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。